In [0]:
! pip uninstall -y tensorflow
! pip install -q tf-nightly
! pip install -q tensorflow-model-optimization

Uninstalling tensorflow-2.2.0rc4:
  Successfully uninstalled tensorflow-2.2.0rc4
     |████████████████████████████████| 521.7MB 27kB/s 
     |████████████████████████████████| 2.9MB 46.6MB/s 
     |████████████████████████████████| 460kB 47.8MB/s 
     |████████████████████████████████| 174kB 5.0MB/s 
     |████████████████████████████████| 296kB 14.8MB/s 


In [0]:
import tempfile
import os

import tensorflow as tf

from tensorflow import keras

In [0]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
#model = keras.Sequential([
#  keras.layers.InputLayer(input_shape=(28, 28)),
#  keras.layers.Reshape(target_shape=(28, 28, 1)),
#  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation=tf.nn.relu),
#  keras.layers.MaxPooling2D(pool_size=(2, 2)),
#  keras.layers.Flatten(),
#  keras.layers.Dense(10, activation=tf.nn.softmax)
#])



model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),   
  keras.layers.Flatten(),                    
  keras.layers.Dense(32, activation=tf.nn.relu),
  keras.layers.Dense(16, activation=tf.nn.relu),
  keras.layers.Dense(10, activation=tf.nn.softmax)
])

# Train the digit classification model
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.05),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=5,
  validation_split=0.1,
)
model.summary()

11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.4711 - accuracy: 0.8585 - val_loss: 0.2143 - val_accuracy: 0.9413
Epoch 2/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.2177 - accuracy: 0.9354 - val_loss: 0.1666 - val_accuracy: 0.9535
Epoch 3/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1678 - accuracy: 0.9499 - val_loss: 0.1440 - val_accuracy: 0.9615
Epoch 4/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1403 - accuracy: 0.9582 - val_loss: 0.1269 - val_accuracy: 0.9642
Epoch 5/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1221 - accuracy: 0.9629 - val_loss: 0.1288 - val_accuracy: 0.9623
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
__

In [0]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

q_aware_model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer_1 (QuantizeLa (None, 28, 28)            3         
_________________________________________________________________
quant_flatten (QuantizeWrapp (None, 784)               1         
_________________________________________________________________
quant_dense (QuantizeWrapper (None, 32)                25125     
_________________________________________________________________
quant_dense_1 (QuantizeWrapp (None, 16)                533       
_________________________________________________________________
quant_dense_2 (QuantizeWrapp (None, 10)                175       
Total params: 25,837
Trainable params: 25,818
Non-trainable params: 19
_________________________________________________________________


In [0]:
# Check Accuracy after Fiune Tuning
train_images_subset = train_images[0:1000] # out of 60000
train_labels_subset = train_labels[0:1000]

q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=50, epochs=5, validation_split=0.1)


Epoch 1/5
18/18 [==============================] - 0s 10ms/step - loss: 0.2755 - accuracy: 0.9267 - val_loss: 0.2372 - val_accuracy: 0.9400
Epoch 2/5
18/18 [==============================] - 0s 4ms/step - loss: 0.1413 - accuracy: 0.9767 - val_loss: 0.2000 - val_accuracy: 0.9500
Epoch 3/5
18/18 [==============================] - 0s 4ms/step - loss: 0.0973 - accuracy: 0.9856 - val_loss: 0.1872 - val_accuracy: 0.9600
Epoch 4/5
18/18 [==============================] - 0s 4ms/step - loss: 0.0684 - accuracy: 0.9911 - val_loss: 0.1739 - val_accuracy: 0.9600
Epoch 5/5
18/18 [==============================] - 0s 3ms/step - loss: 0.0516 - accuracy: 0.9944 - val_loss: 0.1900 - val_accuracy: 0.9600


In [0]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

Baseline test accuracy: 0.9589999914169312
Quant test accuracy: 0.9527000188827515


In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter = True
quantized_tflite_model = converter.convert()

In [0]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy


In [0]:
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Quant TFLite test_accuracy:', test_accuracy)
print('Quant TF test accuracy:', q_aware_model_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Quant TFLite test_accuracy: 0.9478
Quant TF test accuracy: 0.9527000188827515


In [0]:
from google.colab import files
# Create float TFLite model.
float_converter = tf.lite.TFLiteConverter.from_keras_model(model)
float_tflite_model = float_converter.convert()

# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')

with open(quant_file, 'wb') as f:
  f.write(quantized_tflite_model)

with open(float_file, 'wb') as f:
  f.write(float_tflite_model)

print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))

Float model in Mb: 0.10034942626953125
Quantized model in Mb: 0.0278167724609375


In [0]:
files.download(quant_file)